In [2]:
using DifferentiableStateSpaceModels, LinearAlgebra, Turing, Zygote
using DifferentiableStateSpaceModels.Examples
using Turing: @addlogprob!
Turing.setadbackend(:zygote)

# Create models from modules and then solve
model_rbc = @include_example_module(Examples.rbc_observables)

# Generate artificial data for estimation
p_f = (ρ=0.2, δ=0.02, σ=0.01, Ω_1=0.01) # Fixed parameters
p_d = (α=0.5, β=0.95) # Pseudo-true values
sol = generate_perturbation(model_rbc, p_d, p_f, Val(1))
sol_second = generate_perturbation(model_rbc, p_d, p_f, Val(2))

T = 20
ϵ = [randn(model_rbc.n_ϵ) for _ in 1:T]
x0 = zeros(model_rbc.n_x)
fake_z = solve(sol, x0, (0, T), DifferentiableStateSpaceModels.LTI(); noise = ϵ).z
fake_z_second = solve(sol_second, x0, (0, T), DifferentiableStateSpaceModels.QTI(); noise = ϵ).z

┌ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
└ @ Base loading.jl:1342


21-element Vector{Vector{Float64}}:
 [7.824904812740593e-5, 0.0]
 [0.009388280821124171, -7.824904812740593e-5]
 [0.004141003230806073, 0.08564547473108383]
 [0.003113377412500409, 0.043761814038505646]
 [0.007700917546593067, 0.03114748946994769]
 [0.0028219400485847866, 0.07243491327720691]
 [0.014351652682013355, 0.030627394848069485]
 [0.014965386945969147, 0.1336674064514162]
 [0.014501839251820175, 0.1470204417729682]
 [0.012121007622016115, 0.14375484164268565]
 [0.006158082144511833, 0.12158897809077801]
 [0.0022856243818403987, 0.06503713781067733]
 [-0.005567886073177921, 0.02513421406357805]
 [-0.007159495107005886, -0.05012136636762045]
 [-0.0009141646711710215, -0.07044723363875496]
 [0.006966287246748618, -0.014495428593979284]
 [0.013428142860868263, 0.06225784779844589]
 [0.013777970823039514, 0.1275144960461263]
 [0.009452327892967818, 0.13562318681100563]
 [0.009420130597109135, 0.09641143723837649]
 [0.0071400525559970375, 0.09316115727178581]

In [3]:
## Estimation example: first-order, marginal likelihood approach

# Turing model definition
@model function rbc_kalman(z, m, p_f, cache)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, sol.x_ergodic, (0, length(z)); observables = z).logpdf
end

c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_kalman(fake_z, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
chain = sample(turing_model, NUTS(n_adapts, δ), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:42


Chains MCMC chain (1000×14×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 121.72 seconds
Compute duration  = 121.72 seconds
parameters        = α, β
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4584    0.0338     0.0011    0.0021   237.0054    0.9998     ⋯
           β    0.9516    0.0144     0.0005    0.0009   212.9579    1.0014     ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

       

In [4]:
## Estimation example: first-order, joint likelihood approach

# Turing model definition
@model function rbc_joint(z, m, p_f, cache, x0 = zeros(m.n_x))
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    T = length(z)
    ϵ_draw ~ MvNormal(T, 1.0)
    ϵ = map(i -> ϵ_draw[((i-1)*m.n_ϵ+1):(i*m.n_ϵ)], 1:T)
    # println(p_d)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, x0, (0, T); noise = ϵ, observables = z).logpdf
end

c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_joint(fake_z, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5 # A lower max_depth will lead to higher autocorrelation of samples, but faster. The time complexity is approximately 2^max_depth
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.21250000000000002
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:02:26


Chains MCMC chain (1000×35×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 182.9 seconds
Compute duration  = 182.9 seconds
parameters        = α, β, ϵ_draw[1], ϵ_draw[2], ϵ_draw[3], ϵ_draw[4], ϵ_draw[5], ϵ_draw[6], ϵ_draw[7], ϵ_draw[8], ϵ_draw[9], ϵ_draw[10], ϵ_draw[11], ϵ_draw[12], ϵ_draw[13], ϵ_draw[14], ϵ_draw[15], ϵ_draw[16], ϵ_draw[17], ϵ_draw[18], ϵ_draw[19], ϵ_draw[20], ϵ_draw[21]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4581    0.0303     0.0010    0.0029    88.5694    1.0073     ⋯
           β    0.9530    0.0129     0.0004    0.0010   137.1778    

In [5]:
## Estimation example: second-order, joint likelihood approach

# Turing model definition
@model function rbc_second(z, m, p_f, cache, x0 = zeros(m.n_x))
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (α = α, β = β)
    T = length(z)
    ϵ_draw ~ MvNormal(T, 1.0)
    ϵ = map(i -> ϵ_draw[((i-1)*m.n_ϵ+1):(i*m.n_ϵ)], 1:T)
    sol = generate_perturbation(m, p_d, p_f, Val(2); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, x0, (0, T); noise = ϵ, observables = z).logpdf
end

c = SolverCache(model_rbc, Val(2), p_d)
turing_model = rbc_second(fake_z_second, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.3361328125
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:04:48


Chains MCMC chain (1000×35×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 314.0 seconds
Compute duration  = 314.0 seconds
parameters        = α, β, ϵ_draw[1], ϵ_draw[2], ϵ_draw[3], ϵ_draw[4], ϵ_draw[5], ϵ_draw[6], ϵ_draw[7], ϵ_draw[8], ϵ_draw[9], ϵ_draw[10], ϵ_draw[11], ϵ_draw[12], ϵ_draw[13], ϵ_draw[14], ϵ_draw[15], ϵ_draw[16], ϵ_draw[17], ϵ_draw[18], ϵ_draw[19], ϵ_draw[20], ϵ_draw[21]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4464    0.0365     0.0012    0.0048    21.0310    1.0559     ⋯
           β    0.9576    0.0163     0.0005    0.0020    23.2836    